# MChem Project Programme

In [1]:
from rdkit import Chem
from rdkit import RDConfig
from rdkit.Chem.rdMolDescriptors import GetUSRCAT, GetUSRScore
from rdkit.Chem import Descriptors, AllChem, Draw, PandasTools, rdMolAlign
import pandas as pd
import os
import numpy as np
import pubchempy as pcp #can remove this import + the one below - dont need if working with SDF!
from pubchempy import Compound #Requires ADDITIONAL installation into TeachOpenCADD:
#"conda install -c mcs07 pubchempy"
#connects to Pubchem database (used list of pubchem ids (cids))

RDKit WARNING: [13:58:58] Enabling RDKit 2019.09.3 jupyter extensions


### Generation of sdf file from Pubchem .csv database file (Step 1)

<font color = red> If we're working with the dataset I originally generated this section is not needed - just use the SDF i sent a while back! </font>

In [2]:
#Initial .csv input is downloaded directly from PubChem after doing an initial 2D filter 
#using Tanimoto Coefficient set at 70% - Total Compounds at this stage: 649, 166

In [3]:
data = pd.read_csv("PubChem_2D_Filtered_Database.csv") #read in csv to df
data

/Users/jonathan/anaconda3/envs/teachopencadd/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,14,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cid,cmpdname,cmpdsynonym,mw,mf,polararea,complexity,xlogp,heavycnt,hbonddonor,hbondacc,rotbonds,inchikey,iupacname,meshheadings,annothits,annothitcnt,aids,cidcdate,dois
0,801,1H-Indol-3-ylacetate,3-indoleacetate|1H-indol-3-ylacetate|auxin|3-I...,174.18,C10H8NO2-,55.9,200.0,2.1,13,1,2,1,SEOVTRFCIGRIMH-UHFFFAOYSA-M,2-(1H-indol-3-yl)acetate,Indoleacetic Acids,3862.0,7.0,1347041,20040916,/s0716-97602008000200010|/s0716-97602009000400...
1,802,Indole-3-acetic acid,indole-3-acetic acid|87-51-4|3-Indoleacetic ac...,175.18,C10H9NO2,53.1,205.0,1.4,13,2,2,2,SEOVTRFCIGRIMH-UHFFFAOYSA-N,2-(1H-indol-3-yl)acetic acid,NaN,130879.0,15.0,"155,157,161,165,167,175,330,342,422,608,811,83...",20040916,/s0716-97602010000100011|10.1001/archpsyc.1959...
2,1024,Pyrroloquinoline quinone,methoxatin|pyrroloquinoline quinone|72909-34-3...,330.21,C14H6N2O8,175.0,647.0,NaN,24,4,9,3,MMXZSJMASHPLLR-UHFFFAOYSA-N,"4,5-dioxo-1H-pyrrolo[2,3-f]quinoline-2,7,9-tri...",PQQ Cofactor,53086.0,11.0,"1156932,1156933,1156934,1156936",20040916,10.1002/1521-3765(20020916)8:18&lt;4138::aid-c...
3,1372,1-Benzyl-5-methoxy-2-methyl-1h-indol-3-yl)-ace...,CHEMBL148756|1-benzyl-5-methoxy-2-methyl-1h-in...,309.40,C19H19NO3,51.5,407.0,3.7,23,1,3,5,ZEKCBTQHDTUHRJ-UHFFFAOYSA-N,2-(1-benzyl-5-methoxy-2-methylindol-3-yl)aceti...,NaN,34582.0,7.0,"155,157,161,165,167,175,158929,228955",20050325,10.1007/BF00523961|10.1007/s00894-012-1741-4|1...
4,1543,Brequinar analog,BREQUINAR ANALOG|CHEMBL220467|2-biphenyl-4-yl-...,357.40,C23H16FNO2,50.2,513.0,NaN,27,1,4,3,WYKKHJQZENLZID-UHFFFAOYSA-N,6-fluoro-3-methyl-2-(4-phenylphenyl)quinoline-...,NaN,100118.0,8.0,275258,20050325,10.1021/jm0602256|10.1023/A:1015930826903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648577,145944634,NaN,NaN,243.23,C14H10FNO2,47.0,311.0,1.8,18,0,4,4,XHHYRPDZLNJJFW-UHFFFAOYSA-N,"1-(4-fluorophenyl)-3-pyridin-2-ylpropane-1,3-d...",NaN,NaN,NaN,NaN,20200302,NaN
648578,145944657,NaN,NaN,193.24,C11H15NO2,39.2,192.0,NaN,14,0,3,4,NWALSGSGQJFDQW-UHFFFAOYSA-N,methyl 2-(2-methylpropyl)pyridine-3-carboxylate,NaN,NaN,NaN,NaN,20200302,NaN
648579,145944836,NaN,NaN,353.80,C21H20ClNO2,30.2,385.0,NaN,25,0,3,6,SFBHVYWZWRWWQY-UHFFFAOYSA-M,benzyl 1-benzyl-6-methylpyridin-1-ium-3-carbox...,NaN,NaN,NaN,NaN,20200302,NaN
648580,145944837,NaN,NaN,318.40,C21H20NO2+,30.2,385.0,NaN,24,0,2,6,DDXKIXLBXHIXQP-UHFFFAOYSA-N,benzyl 1-benzyl-6-methylpyridin-1-ium-3-carbox...,NaN,NaN,NaN,NaN,20200302,NaN


In [4]:
cids = list(data['cid']) #Create list of PubChem IDs

In [ ]:
smiles = [] #Get SMILEs from PubChem using PubChem ID
for cid in cids:
    c = Compound.from_cid(cid)
    record = Compound.to_series(c)
    smiles.append(record['canonical_smiles'])
    
#cid -> SMILEs -> SDF (RDKit)

In [ ]:
mol_list = [Chem.MolFromSmiles(smile) for smile in smiles] #Generate 2D mols from SMILEs

for mol in mol_list:
    AllChem.EmbedMolecule(mol) #Generate 3D Co-ordinates

In [ ]:
initialset = pd.DataFrame()
initialset['CID'] = cids
initialset['smiles'] = smiles
initialset['Mol'] = mol_list

In [ ]:
#Save structures to SDF
PandasTools.WriteSDF(testset, 'PubChem_2D_Filtered_Database.sdf', molColName='Mol', idName='CID', properties=None, allNumeric=False)

In [ ]:
initialset

In [ ]:
initialset = PandasTools.LoadSDF('PubChem_2D_Filtered_Database.sdf')

In [ ]:
initialset

### GetUSRCAT for .sdf provided (Step 2)

<font color = red> the two boxes directly underneath this are doing the same thing! The first of the two boxes does this absolutely fine and in a more efficient way (shown by the moltomolblock print) - you can remove the second box! </font>

In [ ]:
#keep
QK9mols = Chem.SDMolSupplier('9QK_Ideal.sdf')
m1 = QK9mols[0]

In [ ]:
## the above box loads the query molecule - you do not need the box below!!
print(Chem.MolToMolBlock(m1))

In [ ]:
m0_3D = Chem.AddHs(m1) #you're actually using m1 anyway not the Querydf!!
AllChem.EmbedMolecule(m0_3D)
AllChem.UFFOptimizeMolecule(m0_3D) # Improves the quality of the conformation; this step should not be necessary since v2018.09: default conformations use ETKDG
Draw.MolToImage(m0_3D)

In [ ]:
m0_mom = GetUSRCAT(m0_3D)

<font color = red> As long as the "PubChem_2D_Filtered_Database.sdf is in the same folder as this notebook on your laptop the single line of code below loads the dataframe - again the second box is not needed! Instead add two lines of code (which i have done): (1) fulldf (displays the dataframe) (2) mols = list(fulldf['ROMol']) generates a list of molecules from the dataframe, allowing you to generate the USRCAT moments <font/>

In [ ]:
#sdfFile = os.path.join(RDConfig.RDDataDir,'Filtered_Data.sdf') 
fulldf = PandasTools.LoadSDF('PubChem_2D_Filtered_Database.sdf')

fulldf

#the .sdf file must be the in following directory to be referenced correctly:
#"/Users/[your profile]/anaconda3/envs/teachopencadd/share/RDKit/Data/Filtered_Data.sdf"

In [ ]:
#takes the ROMol column of the df and makes a list which can be used to 
mols = list(fulldf['ROMol'])

In [ ]:
USRCAT_Mom = []
for i in range(len(mols)):
    mom = GetUSRCAT(mols[i])
    USRCAT_Mom.append(mom)

In [ ]:
USRCAT_Score = []
for i in range(len(USRCAT_Mom)):
    Score = GetUSRScore(m0_mom, USRCAT_Mom[i])
    USRCAT_Score.append(Score)

In [ ]:
fulldf['USRCAT_Mom'] = USRCAT_Mom 

In [ ]:
fulldf['USRCAT_Score'] = USRCAT_Score

In [ ]:
fulldf

In [ ]:
fulldf = fulldf.sort_values('USRCAT_Score', ascending=False)

In [ ]:
fulldf = fulldf.reset_index(drop = True)
#resets the index so the dataframe is updated with high-to-low values of USRCAT_Score, 0 being the highest and 9 the lowest.

In [ ]:
fulldf

<font color = red> add the following 2 steps to create a new sdf containing the top 5 results from our USRCAT similarity comparison: (1) subdf = fulldf[0:5] creates a new dataframe which contains a subset of fulldf (2) add a line of code to write the new dataframe to a sdf (hint: you have a line which does this in section 1 - just be careful when changing the variables it takes in, make sure you use subdf!) <font/>

In [ ]:
subdf = fulldf[0:5]

### Alignment of Co-Ordinates of Query Molecule for OpenMM studies (Step 3)

<font color = red> don't need to load the query molecule again here - we've done this above already (named m1 - just refer to this!) for this section to work, need a list with the query in position 0 and our 5 top results appended after. I would however use the Chem.SDMolSupplier method to load the 5 molecules here as you have already done, we don't need a dataframe! make sure you change the name of the file here - at the moment this is loading all 3000+ molecules!! <font/>

In [ ]:
#keep
mols = []
mols.append(m1)

In [ ]:
print(Chem.MolToMolBlock(mols[0]))

In [ ]:
#keep - but make sure to change the filename to whatever SDF you write the top 5 to!
suppl = Chem.SDMolSupplier('PubChem_2D_Filtered_Database.sdf')
for mol in suppl:
    mols.append(mol) #good - this appends to the end of our mols list
    
    #SDF of the query molecule (for molecular alignment you want a list of molecules with a 
    #Query in first poisiton and the rest 1 -> n)


<font color = red> Everything from here down is fine! the print of the molblock between pdb files is nice but not 100% needed - can remove to tidy up if you want! <font/>

In [ ]:
for probeMol in mols[1:]:
    print(Chem.MolToMolBlock(probeMol))

In [ ]:
Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(200, 200), legends=None, highlightAtomLists=None, highlightBondLists=None, useSVG=False)

In [ ]:
#This is the section which calculates the alignment
refMol = mols[0] #We would set this to the ligand already bound to the protein (query)
score_list = []
for probeMol in mols[1:]:
    pyO3A = rdMolAlign.GetO3A(probeMol, refMol)
    score = pyO3A.Align()
    score_list.append(score)
print(score_list)

In [ ]:
Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(200, 200), legends=None, highlightAtomLists=None, highlightBondLists=None, useSVG=False)

#Printed twice to show that the coordinates have changed and the alignment has worked correctly.

In [ ]:
for probeMol in mols[1:]:
    print(Chem.MolToMolBlock(probeMol))

In [ ]:
fin = open("5OBJ-1.pdb", "w")
    probeMol = mols[1]
    pdbblock = Chem.MolToPDBBlock(probeMol)
    fin.write(pdbblock)
fin.close()

In [ ]:
#the prints inbetween are nice but not neccessary - can remove if you want!
for probeMol in mols[2:]:
    print(Chem.MolToMolBlock(probeMol))

In [ ]:
fin = open("5OBJ-2.pdb", "w")
    probeMol = mols[2]
    pdbblock = Chem.MolToPDBBlock(probeMol)
    fin.write(pdbblock)
fin.close()

In [ ]:
for probeMol in mols[3:]:
    print(Chem.MolToMolBlock(probeMol))

In [ ]:
fin = open("5OBJ-3.pdb", "w")
    probeMol = mols[3]
    pdbblock = Chem.MolToPDBBlock(probeMol)
    fin.write(pdbblock)
fin.close()

In [ ]:
for probeMol in mols[4:]:
    print(Chem.MolToMolBlock(probeMol))

In [ ]:
fin = open("5OBJ-4.pdb", "w")
    probeMol = mols[4]
    pdbblock = Chem.MolToPDBBlock(probeMol)
    fin.write(pdbblock)
fin.close()

In [ ]:
for probeMol in mols[5:]:
    print(Chem.MolToMolBlock(probeMol))

In [ ]:
fin = open("5OBJ-5.pdb", "w")
    probeMol = mols[5]
    pdbblock = Chem.MolToPDBBlock(probeMol)
    fin.write(pdbblock)
fin.close()